In [1]:
# Imports
import pandas as pd
import numpy as np


In [3]:
# Diagnosis for each admission to hospital. This is from MIMIC III. Data needs to be obtained through MIMIC III, and not included here. 
df_diagcode = pd.read_csv('DIAGNOSES_ICD.csv')

In [4]:
df_diagcode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
ROW_ID        651047 non-null int64
SUBJECT_ID    651047 non-null int64
HADM_ID       651047 non-null int64
SEQ_NUM       651000 non-null float64
ICD9_CODE     651000 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


In [6]:
print('There are {} unique ICD9 codes in MIMIC III dataset.'.format(df_diagcode['ICD9_CODE'].value_counts().count()))
print('Dataset has {} number of unique patients.'.format(df_diagcode['SUBJECT_ID'].nunique()))
print('Dataset has {} number of unique hospital admissions.'.format(df_diagcode['HADM_ID'].nunique()))

There are 6984 unique ICD9 codes in MIMIC III dataset.
Dataset has 46520 number of unique patients.
Dataset has 58976 number of unique hospital admissions.


In [7]:
df_diagcode = df_diagcode.dropna(subset=['ICD9_CODE'])

In [8]:
df_diagcode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651000 entries, 0 to 651046
Data columns (total 5 columns):
ROW_ID        651000 non-null int64
SUBJECT_ID    651000 non-null int64
HADM_ID       651000 non-null int64
SEQ_NUM       651000 non-null float64
ICD9_CODE     651000 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 29.8+ MB


In [9]:
# Filter out E and V codes since processing will be done on the numeric first 3 values
df_diagcode['code'] = df_diagcode['ICD9_CODE']
df_diagcode['code'] = df_diagcode['code'][~df_diagcode['code'].str.contains("[a-zA-Z]").fillna(False)]
df_diagcode['code'].fillna(value='-999999', inplace=True)

# https://stackoverflow.com/questions/46168450/replace-specific-range-of-values-in-data-frame-pandas
df_diagcode['code'] = df_diagcode['code'].str.slice(start=0, stop=3, step=1)
df_diagcode['code'] = df_diagcode['code'].astype(int)

In [10]:
print(df_diagcode)

        ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE  code
0         1297         109   172335      1.0     40301   403
1         1298         109   172335      2.0       486   486
2         1299         109   172335      3.0     58281   582
3         1300         109   172335      4.0      5855   585
4         1301         109   172335      5.0      4254   425
...        ...         ...      ...      ...       ...   ...
651042  639798       97503   188195      2.0     20280   202
651043  639799       97503   188195      3.0     V5869   -99
651044  639800       97503   188195      4.0     V1279   -99
651045  639801       97503   188195      5.0      5275   527
651046  639802       97503   188195      6.0      5569   556

[651000 rows x 6 columns]


In [11]:
# ICD-9 top-level ranges
icd9_ranges = [(1, 140), (140, 240), (240, 280), (280, 290), (290, 320), (320, 390), 
               (390, 460), (460, 520), (520, 580), (580, 630), (630, 680), (680, 710),
               (710, 740), (740, 760), (760, 780), (780, 800), (800, 1000),(1000, 2000), (-99999,-1)]

# grouping names
diag_dict = {0: 'infectious', 1: 'neoplasms', 2: 'endocrine', 3: 'blood',
             4: 'mental', 5: 'nervous', 6: 'circulatory', 7: 'respiratory',
             8: 'digestive', 9: 'genitourinary', 10: 'pregnancy', 11: 'skin', 
             12: 'muscular', 13: 'congenital', 14: 'prenatal', 15: 'symptoms',
             16: 'injury', 17: 'misc', 18: 'E and V'}

# re-code in terms of integer
for num, cat_range in enumerate(icd9_ranges):
    df_diagcode['code'] = np.where(df_diagcode['code'].between(cat_range[0],cat_range[1]), 
            num, df_diagcode['code'])
    
# Convert integer to grouping
df_diagcode['code'] = df_diagcode['code']
df_diagcode['cat'] = df_diagcode['code'].replace(diag_dict)

In [12]:
# Verify
df_diagcode.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,code,cat
0,1297,109,172335,1.0,40301,6,circulatory
1,1298,109,172335,2.0,486,7,respiratory
2,1299,109,172335,3.0,58281,9,genitourinary
3,1300,109,172335,4.0,5855,9,genitourinary
4,1301,109,172335,5.0,4254,6,circulatory


In [13]:
hadm_list = df_diagcode.groupby('HADM_ID')['cat'].apply(list).reset_index()
hadm_list.head()

,HADM_ID,cat
0,100001,"[endocrine, nervous, genitourinary, digestive,..."
1,100003,"[digestive, blood, infectious, digestive, circ..."
2,100006,"[respiratory, respiratory, respiratory, neopla..."
3,100007,"[digestive, digestive, injury, respiratory, ci..."
4,100009,"[circulatory, injury, circulatory, endocrine, ..."


In [15]:
count = df_diagcode.groupby('code').count()

In [17]:
hadm_ICD9_list = df_diagcode.groupby('HADM_ID')['ICD9_CODE'].apply(list).reset_index()
hadm_ICD9_list.head()

,HADM_ID,ICD9_CODE
0,100001,"[25013, 3371, 5849, 5780, V5867, 25063, 5363, ..."
1,100003,"[53100, 2851, 07054, 5715, 45621, 53789, 4019,..."
2,100006,"[49320, 51881, 486, 20300, 2761, 7850, 3090, V..."
3,100007,"[56081, 5570, 9973, 486, 4019]"
4,100009,"[41401, 99604, 4142, 25000, 27800, V8535, 4148..."


In [18]:
# Convert diagnoses list into matrix
hadm_item = pd.get_dummies(hadm_list['cat'].apply(pd.Series).stack()).sum(level=0)
hadm_item.head()

,E and V,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,symptoms
0,2,0,2,0,2,5,2,0,0,0,0,0,2,0,0,0,1,0
1,0,1,2,0,4,0,0,1,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,1,0,0,0,1,0,1,0,0,0,3,0,1
3,0,0,1,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0
4,6,1,7,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0


In [21]:
for i in hadm_item.columns[0:]:
    # make sure there is only 0 & 1 values
    s = hadm_item[i]
    s[s > 1] = 1

In [25]:
hadm_item.head()

,E and V,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,symptoms
0,1,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0
1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1
3,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
4,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [26]:
# Join back with HADM_ID
hadm_item = hadm_item.join(hadm_list['HADM_ID'], how="outer")
hadm_item.head()

,E and V,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,symptoms,HADM_ID
0,1,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,100001
1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,100003
2,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,100006
3,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,100007
4,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,100009


In [28]:
hadm_item.to_csv('18_label.csv',index=False)